In [1]:
# changing in order to not reset paths
%cd -q ../../project/parallel_synthesis/experiments 

from copy import copy
from protera_stability.config.lazy import LazyCall as L
from protera_stability.config.instantiate import instantiate
from protera_stability.config.common.data import base_dataset, base_dataloader, base_sampler, get_train_val_indices


# Does our LazyConfig work?

In [2]:
dataset = instantiate(base_dataset.data)

In [3]:
train_idx, valid_idx = get_train_val_indices(
    dataset,
    0.8
)

assert len(set(train_idx).intersection(set(valid_idx))) == 0

In [4]:
base_sampler.name # this should change for each experiment!

''

## *No Sampler* 
> this is because we aren't using a "special" sampling method, therefore we directly pass the indices

In [5]:
a_sampler = copy(base_sampler)
a_sampler.random.indices = train_idx

a_sampler = instantiate(a_sampler.random)

len(a_sampler), len(train_idx)

(6564, 6564)

## Instantiate Diversity Sampler

In [27]:
div_sampler = copy(base_sampler)
div_sampler.diversity.set_sequences.dataset = dataset.data
div_sampler.diversity.set_sequences.set_indices = train_idx
div_sampler.diversity.max_size = int(len(dataset) * 0.8)

diversity_sampler = instantiate(div_sampler.diversity)

assert len(set(diversity_sampler.indices).intersection(set(valid_idx))) == 0

len(diversity_sampler), diversity_sampler.stopped_by

AttributeError: 'ProteinStabilityDataset' object has no attribute 'data'

## Instantiate Random Sampler

In [7]:
rand_sampler = copy(base_sampler)
rand_sampler.random.indices.dataset = dataset
rand_sampler.random.indices.set_indices = train_idx

random_sampler = instantiate(rand_sampler.random)

assert len(set(random_sampler.indices).intersection(set(valid_idx))) == 0

len(random_sampler), len(dataset) * 0.3, len(train_idx)


(2461, 2461.2, 6564)

### Validation Sampler

In [8]:
val_sampler = copy(base_sampler)
val_sampler = sampler.random.indices = valid_idx
valid_sampler = instantiate(val_sampler.random)

len(valid_sampler), len(valid_idx)

(1640, 1640)

## Instantiate Dataloaders

In [10]:
dataloader.train.sampler = div_sampler.diversity
(instantiate(dataloader.train))

Global seed set to 3232581377


In [11]:
dataloader.valid.sampler = val_sampler.random
instantiate(dataloader.valid)

In [12]:
instantiate(dataloader.test)

In [15]:
len(instantiate(dataloader.train))

Global seed set to 3232581377


24

In [26]:
samples=0
for X, y in instantiate(dataloader.test):
    samples += X.shape[0]

samples

2077